In [1]:
import types
import abc
import sys
import operator

def create_stock_class():
    # Define methods for the class
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

    def cost(self):
        return self.shares * self.price

    # Dictionary of class attributes
    cls_dict = {
        '__init__': __init__,
        'cost': cost,
    }
    
    # Create a new class dynamically
    Stock = types.new_class('Stock', (), {}, lambda ns: ns.update(cls_dict))
    Stock.__module__ = __name__  # Set the module attribute
    return Stock

Stock = create_stock_class()
s = Stock('ACME', 50, 91.1)
print(s.cost())  # Output: 4555.0

# Example using a metaclass
def create_abstract_stock_class():
    cls_dict = {
        '__init__': lambda self, name, shares, price: setattr(self, 'data', (name, shares, price)),
        'cost': lambda self: self.data[1] * self.data[2],
    }
    
    AbstractStock = types.new_class('AbstractStock', (), {'metaclass': abc.ABCMeta}, lambda ns: ns.update(cls_dict))
    AbstractStock.__module__ = __name__
    return AbstractStock

AbstractStock = create_abstract_stock_class()
print(type(AbstractStock))  # Output: <class 'abc.ABCMeta'>

# Named tuple-like structure
def named_tuple(classname, fieldnames):
    cls_dict = {name: property(operator.itemgetter(n)) for n, name in enumerate(fieldnames)}
    
    def __new__(cls, *args):
        if len(args) != len(fieldnames):
            raise TypeError(f'Expected {len(fieldnames)} arguments')
        return tuple.__new__(cls, args)
    
    cls_dict['__new__'] = __new__
    
    cls = types.new_class(classname, (tuple,), {}, lambda ns: ns.update(cls_dict))
    cls.__module__ = sys._getframe(1).f_globals['__name__']  # Set caller module
    return cls

Point = named_tuple('Point', ['x', 'y'])
p = Point(4, 5)
print(p.x, p.y)  # Output: 4 5


4555.0
<class 'abc.ABCMeta'>
4 5
